In [1]:
######################################### IMPORTING PACAKGES #############################
from scipy import spatial
import pandas as pd
import os
import json
import numpy as np
import string
import warnings
warnings.filterwarnings("ignore")
import sys  
import os
from dateutil.parser import parse
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# PDF text extraction
from pdfminer3.layout import LAParams, LTTextBox
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import PDFPageAggregator
from pdfminer3.converter import TextConverter
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
# Others
import requests
import string
import re
from pprint import pprint
from tqdm.notebook import tqdm
import io
import nltk
from nltk.stem.snowball import SnowballStemmer
import spacy
# spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm", disable=['ner'])
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from sklearn.feature_extraction import text
stop_words = text.ENGLISH_STOP_WORDS.union(stop_words)
import pickle #pickle模块
import tensorflow as tf
from utils import extract_pdf,extract_pages_sentences,keyword_filter,custom_standardization,removeStopWords,stemSentence

In [2]:
def predict(pdf_file,model,vectorizer):
    text = extract_pdf(pdf_file)
    pages_content, pages_sentences = extract_pages_sentences(spacy.load("en_core_web_sm"),text)
    df = keyword_filter(pages_sentences,['biodiversity','green space','program','animal','fish','bird','avian','tree','forest','coastal','beach','shoreline',
                                         'clean-up','specie','ecosystem','system','project','protection','conservation','natural resources','wildlife','habitat'])
    df = df.reset_index()
    df['sentence1'] = df['sentence'].apply(lambda x:tf.compat.as_str_any(custom_standardization(x).numpy()))
    df['sentence1'] = df['sentence1'].apply(lambda x:removeStopWords(str(x)))
    df['sentence1'] = df['sentence1'].apply(lambda x:stemSentence(str(x)))
    X = vectorizer.transform(df['sentence'])
    res = model.predict(X)
    sentences = list(df[res == 1]['sentence'])
    if len(sentences)>0:
        label = 'Yes'
    else:
        label = 'False'
    return label, sentences

In [7]:
file = 'ubm_esg_report_2021.pdf'
with open('A8_model_randomforest.pkl', 'rb') as f:
    model = pickle.load(f)
with open('vectorizer_8.pkl', 'rb') as f:
    vectorizer = pickle.load( f)

In [8]:
predict(file,model,vectorizer)

('Yes',
 ['Environmental risks Environmental risks and their impact are becoming increas-ingly important for the planning and realisation of develop-ment projects.',
  'We also exchanged an area of 24,290 m on four brownfield projects.',
  'For us, that means designing projects to minimise the poten-tial negative effects on the local area, for example caused by shading, artificial light, noise, emission and increased traffic, or to more than offset these effects through positive changes and improvements.'])